In [4]:
import pandas as pd

class MarketData:
    def __init__(self, df: pd.DataFrame):
        """
        df must have columns: ['date', 'ticker', 'open', 'high', 'low', 'close', 'volume']
        """
        self.df = df.copy()
        self.df['date'] = pd.to_datetime(self.df['date'])
        self.df.set_index(['date', 'ticker'], inplace=True)

    def get_prices(self, kind='close') -> pd.DataFrame:
        return self.df[kind].unstack()

    def get_volume(self) -> pd.DataFrame:
        return self.df['volume'].unstack()

    def get_returns(self, kind='close') -> pd.DataFrame:
        return self.get_prices(kind).pct_change()

    def get_ma(self, window=20) -> pd.DataFrame:
        return self.get_prices().rolling(window).mean()

    
class DataProcessing:
    def clean(self, df: pd.DataFrame) -> pd.DataFrame:
        return df.dropna(how='all').fillna(method='ffill')

    def normalize(self, df: pd.DataFrame) -> pd.DataFrame:
        return (df - df.mean()) / df.std()

    def fill_na(self, df: pd.DataFrame, method='ffill') -> pd.DataFrame:
        return df.fillna(method=method)

    def reshape_to_matrix(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        return df.pivot_table(index='date', columns='ticker', values=column)

    
class FundamentalData:
    def __init__(self, fundamentals_df):
        """
        Must contain columns: ['ticker', 'PER', 'PB', 'ROE', 'NetMargin', 'Growth']
        """
        self.df = fundamentals_df.set_index('ticker')

    def get_per(self):
        return self.df['PER']

    def get_pb(self):
        return self.df['PB']

    def get_roe(self):
        return self.df['ROE']

    def get_margin(self):
        return self.df['NetMargin']

    def get_growth(self):
        return self.df['Growth']

    
class QuantitativeIndicators:
    def __init__(self, prices: pd.DataFrame, returns: pd.DataFrame, volume: pd.DataFrame):
        self.prices = prices
        self.returns = returns
        self.volume = volume

    def compute_momentum(self, window=20):
        return self.prices / self.prices.shift(window) - 1

    def compute_volatility(self, window=20):
        return self.returns.rolling(window).std()

    def compute_avg_volume(self, window=5):
        return self.volume.rolling(window).mean()

    def compute_relative_price(self, ma_window=20):
        return self.prices / self.prices.rolling(ma_window).mean() - 1

    
class Stock:
    def __init__(self, ticker: str, prices: pd.Series, returns: pd.Series = None):
        self.ticker = ticker
        self.prices = prices
        self.returns = returns if returns is not None else prices.pct_change()
        self.scores = {}

    def update_score(self, score_name: str, value: float):
        self.scores[score_name] = value

    def get_metric(self, metric: str):
        return self.scores.get(metric, None)


In [1]:
import pandas as pd
import numpy as np

# === MarketData ===
class MarketData:
    def __init__(self, df: pd.DataFrame):
        """
        df must have columns: ['date', 'ticker', 'open', 'high', 'low', 'close', 'volume']
        """
        self.df = df.copy()
        self.df['date'] = pd.to_datetime(self.df['date'])
        self.df.set_index(['date', 'ticker'], inplace=True)

    def get_prices(self, kind='close') -> pd.DataFrame:
        return self.df[kind].unstack()

    def get_volume(self) -> pd.DataFrame:
        return self.df['volume'].unstack()

    def get_returns(self, kind='close') -> pd.DataFrame:
        return self.get_prices(kind).pct_change()

    def get_ma(self, window=20) -> pd.DataFrame:
        return self.get_prices().rolling(window).mean()


# === DataProcessing ===
class DataProcessing:
    def clean(self, df: pd.DataFrame) -> pd.DataFrame:
        return df.dropna(how='all').fillna(method='ffill')

    def normalize(self, df: pd.DataFrame) -> pd.DataFrame:
        return (df - df.mean()) / df.std()

    def fill_na(self, df: pd.DataFrame, method='ffill') -> pd.DataFrame:
        return df.fillna(method=method)

    def reshape_to_matrix(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        return df.pivot_table(index='date', columns='ticker', values=column)


# === FundamentalData ===
class FundamentalData:
    def __init__(self, fundamentals_df):
        """
        Must contain columns: ['ticker', 'PER', 'PB', 'ROE', 'NetMargin', 'Growth']
        """
        self.df = fundamentals_df.set_index('ticker')

    def get_per(self):
        return self.df['PER']

    def get_pb(self):
        return self.df['PB']

    def get_roe(self):
        return self.df['ROE']

    def get_margin(self):
        return self.df['NetMargin']

    def get_growth(self):
        return self.df['Growth']


# === QuantitativeIndicators ===
class QuantitativeIndicators:
    def __init__(self, prices: pd.DataFrame, returns: pd.DataFrame, volume: pd.DataFrame):
        self.prices = prices
        self.returns = returns
        self.volume = volume

    def compute_momentum(self, window=20):
        return self.prices / self.prices.shift(window) - 1

    def compute_volatility(self, window=20):
        return self.returns.rolling(window).std()

    def compute_avg_volume(self, window=5):
        return self.volume.rolling(window).mean()

    def compute_relative_price(self, ma_window=20):
        return self.prices / self.prices.rolling(ma_window).mean() - 1


# === Stock ===
class Stock:
    def __init__(self, ticker: str, prices: pd.Series, returns: pd.Series = None):
        self.ticker = ticker
        self.prices = prices
        self.returns = returns if returns is not None else prices.pct_change()
        self.scores = {}

    def update_score(self, score_name: str, value: float):
        self.scores[score_name] = value

    def get_metric(self, metric: str):
        return self.scores.get(metric, None)


In [3]:
import pandas as pd
#from core.market_data import MarketData
#from core.data_processing import DataProcessing
#from core.fundamental_data import FundamentalData
#from core.quant_indicators import QuantitativeIndicators
#from core.stock import Stock

# === 1. Simulated raw input data
df_prices = pd.DataFrame({
    'date': pd.date_range(start='2024-01-01', periods=5).repeat(2),
    'ticker': ['BOAB', 'ETIT'] * 5,
    'open': [98, 12]*5,
    'high': [100, 13]*5,
    'low': [97, 11]*5,
    'close': [99, 12.5]*5,
    'volume': [2000, 3500]*5,
})

df_fundamentals = pd.DataFrame({
    'ticker': ['BOAB', 'ETIT'],
    'PER': [8.5, 12.0],
    'PB': [1.2, 1.8],
    'ROE': [0.14, 0.10],
    'NetMargin': [0.11, 0.07],
    'Growth': [0.05, 0.04],
})

# === 2. Expected Result Overview
# - Momentum (2-day) for BOAB should be 0 because prices are flat at 99
# - Volatility (2-day std of returns) for BOAB should be 0
# - PER for BOAB is 8.5 from fundamentals

# === 3. Run processing pipeline
market = MarketData(df_prices)
processor = DataProcessing()
cleaned_prices = processor.clean(market.get_prices())
returns = market.get_returns()
volume = market.get_volume()

# Inject corrected indicators
indicators = QuantitativeIndicators(prices=cleaned_prices, returns=returns, volume=volume)
momentum = indicators.compute_momentum(window=2)
volatility = indicators.compute_volatility(window=2)
avg_volume = indicators.compute_avg_volume(window=2)

fundamentals = FundamentalData(df_fundamentals)

# === 4. Build stock and inject scores
stock_boab = Stock(ticker='BOAB', prices=cleaned_prices['BOAB'], returns=returns['BOAB'])
stock_boab.update_score('momentum', momentum['BOAB'].iloc[-1])    # Expect ~0.0
stock_boab.update_score('volatility', volatility['BOAB'].iloc[-1]) # Expect ~0.0
stock_boab.update_score('PER', fundamentals.get_per()['BOAB'])     # Expect 8.5

# === 5. Output with expectations
print("=== BOAB TEST ===")
print(f"Expected Momentum: 0.0   → Got: {stock_boab.get_metric('momentum'):.4f}")
print(f"Expected Volatility: 0.0 → Got: {stock_boab.get_metric('volatility'):.4f}")
print(f"Expected PER: 8.5        → Got: {stock_boab.get_metric('PER')}")


=== BOAB TEST ===
Expected Momentum: 0.0   → Got: 0.0000
Expected Volatility: 0.0 → Got: 0.0000
Expected PER: 8.5        → Got: 8.5


In [4]:
display(df_prices), display(df_fundamentals)

,date,ticker,open,high,low,close,volume
0,2024-01-01,BOAB,98,100,97,99.0,2000
1,2024-01-01,ETIT,12,13,11,12.5,3500
2,2024-01-02,BOAB,98,100,97,99.0,2000
3,2024-01-02,ETIT,12,13,11,12.5,3500
4,2024-01-03,BOAB,98,100,97,99.0,2000
5,2024-01-03,ETIT,12,13,11,12.5,3500
6,2024-01-04,BOAB,98,100,97,99.0,2000
7,2024-01-04,ETIT,12,13,11,12.5,3500
8,2024-01-05,BOAB,98,100,97,99.0,2000
9,2024-01-05,ETIT,12,13,11,12.5,3500


,ticker,PER,PB,ROE,NetMargin,Growth
0,BOAB,8.5,1.2,0.14,0.11,0.05
1,ETIT,12.0,1.8,0.10,0.07,0.04


(None, None)